# 정형 데이터

어떤 구조로 이뤄진 데이터. csv나 데이터베이스, 혹은 엑셀에 저장하기 쉽다.

# 비정형 데이터

데이터베이스나 엑셀로 표현하기 어려운 데이터. 주로 텍스트 데이터, 디지털카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등이 있다.

# 앙상블 학습(ensemble learning)

더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 **머신러닝 알고리즘**.

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘으로 대부분 **결정 트리(DecisionTreeClassifier)**를 기반으로 만들어져있다.



`비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다. 하지만, 신경망 알고리즘을 사용하면 만들 수 있다.`

## 랜덤 포레스트(random forest) - RandomForestClassifier

앙상블 학습의 대표 주자. 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.(결과값을 평균으로 만들어 보여줌.)



- **각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위**로 고른 다음 이 중에서 **최선의 분할을 찾는다**.

- RandomForestClassifier는 기본적으로 **전체 특성 개수의 제곱근만큼의 특성**을 선택한다.(ex - 4개의 특성이 있다면 2개를 랜덤하게 선택하여 사용한다.)

- 다만, **회귀 모델인 RandomForestRegressor는 전체 특성**을 사용한다.(RandomForestClassifier는 분류 모델.)

- 기본적으로 **100개의 결정 트리**를 훈련.

- **분류일 때는 각 트리의 클래스별 확률을 평균**하여 **가장 높은 확률**을 가진 클래스를 예측으로 삼는다.(**회귀일 때는 단순히 각 트리의 예측을 평균**)

- **랜덤**하게 선택한 샘플과 특성을 사용하기 때문에 **훈련 세트의 과대적합을 막아**주고 **검증 세트와 테스트 세트에서 안정적인 성능**을 얻을 수 있다.

- 결정 트리의 큰 장점 중 하나인 **특성 중요도**를 계산할 때, **랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것**이다.


**각 트리**를 훈련하기 위한 데이터를 랜덤하게 만드는데, 입력 받은 훈련 데이터에서 **랜덤하게 샘플을 추출**하여 훈련 데이터를 만든다.(**중복 가능**). 이렇게 만들어진 샘플을 **부트스트랩 샘플(bootstrap sample)**이라고 부른다. 기본적으로 **부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다**. 

### 부트스트랩 샘플(bootstrap sample)

데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식. 결국 부트스트랩 방식으로 샘플링하여 분류한 데이터라는 의미.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)                        # return_train_score=True 일 경우, 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환. 디폴트값은 False.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))                 # 각 트리의 클래스별 확률을 평균.(cross_validate 안의 딕셔너리에 trian_score, test_score가 저장되어있다.)

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)                                                 # 특성 중요도를 측정하기 위해 랜덤 포레스트를 사용하여 훈련을 진행.
print(rf.feature_importances_)                                                    # 결정 트리에서 확인했던 중요도보다 당도의 중요도는 떨어졌고, pH의 중요도는 올랐다.
                                                                                    # 그 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리 훈련을 했기 때문이다.
                                                                                    # (과대적합을 줄이고 일반 성능을 올림.)

[0.23167441 0.50039841 0.26792718]


### OOB 점수(out of bag) - oob_score

랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리 훈련을 하는데, 이때 **부트스트랩 샘플에 포함되지 않고 남아있는 샘플을 OOB 샘플**이라고 한다.

이렇게 남**는 샘플을 이용**하여 부트스트랩 샘플로 훈련한 **결정 트리를 평가할 수 있다.**(검증 세트의 역할이라고 볼 수 있다.)

**OOB 점수**를 사용하면 **교차 검증을 대신**할 수 있어서 결과적으로 **훈련 세트에 더 많은 샘플을 사용할 수 있다**.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)            # oob_score=True 인 경우 결정 트리의 OOB 점수를 평균하여 출력.
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리(Extra Tree) - ExtraTreesClassifier

랜덤 포레스트와 매우 비슷하게 동작하며 **기본적으로 100개의 결정 트리 훈련을 한다**.

결정 트리가 제공하는 대부분의 매개변수를 지원하여 **전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용.**

엑스트라 트리는 **부트스트랩 샘플을 사용하지 않는다.** 즉, **결정 트리를 만들 때 전체 훈련 세트를 사용.**

대신 **노드를** 분할할 때 가장 좋은 분할을 찾는 것이 아니라 **무작위로 분할.**

splitter='random'을 사용.

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만, **많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음.**

보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리 훈련을 해야함.

하지만, 랜덤하게 노드를 분할하기 때문에 **빠른 계산 속도가 엑스트라 트리의 장점.**

엑스트라 트리의 회귀 버전은  ExtraTreesRegressor 클래스이다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)                                                    # 엑스트라 트리도 특성 중요도(feature_importances_) 제공.

[0.20183568 0.52242907 0.27573525]


앞의 5-1의 '결정 트리'에서 만든 '특성 중요도(feature_importnaces_)'와 비교하여 보면 **'랜덤 포레스트', '엑스트라 트리'는 당도에 대한 중요도가 감소하고 pH의 중요도가 증가한 것을 볼 수 있다**.

즉, **특성을 랜덤으로 뽑아서 훈련을 진행했기 때문에 하나의 특성에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여한 것이다.**

여기서 '랜덤 트리'와 '엑스트라 트리'의 차이점은 '엑스트라 트리'가 무작위성이 훨씬 크다는 것인데, 이 모델에서는 특성의 개수가 적기 때문에 큰 차이점을 찾아보기 힘들다.(만약, titanic을 이용한다면 차이점을 확실히 알아볼 수 있을 것이다.) 

## 그레이디언트 부스팅(gradient boosting) - GradientBoostingClassifier

**깊이가 얕은** 결정 트리를 사용하여 **이전 트리의 오차를 보안하는 방식**.

기본적으로 **깊이가 3인 결정 트리를 100개 사용**.

**과대적합에 강하고** 일반적으로 **높은 일반화 성능을 기대할 수 있음**.

**경사 하강법을 사용**하여 트리 앙상블에 추가.

(**분류에서는 로지스특 손실 함수**, **회귀에서는 평균 제곱 오차 함수를 사용**.)

**학습률 매개변수로 속도를 조절.**

결정 트리의 개수를 늘려도 **과대적합에 매우 강함.**

**학습률**을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)                                   # GradientBoostingClassifier에는 n_jobs가 없다.
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


'랜덤 트리', '엑스트라 트리'와는 달리 과대적합이 해소되어 나오게 된다.

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,               # n_estimators : 결정 트리의 개수를 지정한다.(디폴트값은 100)
                                random_state=42)                                   # learning_rate(학습률) : 디폴트값은 0.1

scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


n_estimateors로 결정 트리의 개수를 500개(디폴트값의 5배)나 늘렸지만, 과대적합을 잘 억제할 정도로 과대적합에 강하다.

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그래이디언트 부스팅은 경사 하강법을 기본으로 사용하고 있어서 일부 특성(당도)에 집중하는 모습을 볼 수 있다.

즉, 경사 하강법을 사용하여 결정 트리의 개수를 500개로 올리니 '랜덤 트리', '엑스트라 트리'에서와 달리 당도의 중요도가 올라갔으며, pH의 중요도는 다시 떨어졌다.

그래이디언트 부스팅은 훈련률을 조절할 뿐만 아니라 **훈련 세트의 비율**을 정하는 **subsample** 매개변수를 사용할 수 있다.(디폴트값은 1.0) 

하지만 **1보다 작으면 훈련 세트의 일부만을 사용**한다.

이것은 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷하다.

일반적으로 그래이디언트 부스팅은 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다. 그러나 **순서대로 트리를 추가하기 때문에 훈련 속도가 느리다**.

즉, GradientBoostingClassifier에는 **n_jobs**(코어 사용 수를 지정)매개변수가 **없다**는 뜻이다.

회귀 버전은 GradientBoostingRegressor이다.

여기까지는 sklearn을 사용.

____________________________

sklearn이 아닌 다른 패키지를 사용.

## 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting) - HistGradientBoostingClassifier

**정형 데이터를 다루는 머신러닝 알고리즘**.

- **입력 특성을 256개의 구간**으로 나눈다. 따라서 **노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.**

- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용. -> **누락된 특성이 있더라도 전처리가 필요 없다.**

- 히스토그램 기반 그레이디언트 부스팅은 기본 매개변수에서 **안정적인 성능**을 얻을 수 있다.

- 트리의 개수를 지정하는 n_estimators 대신에 **부스팅 반복 횟수를 지정하는 max_iter**를 사용한다.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))               # 히스토램 기반 그레이디언트 부스팅도 score를 구하기 위서는 mean을 해야한다.

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면 그레이디언트 부스팅보다 조금 더 높은 성능을 제공.

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)

result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)          # n_repeats : 랜덤하게 섞을 횟수를 지정.(디폴트값은 5)

print(result.importances); print()
print(result.importances_mean); print()
print(result.importances_std)

[[0.08793535 0.08350972 0.08908986 0.08312488 0.09274581 0.08755051
  0.08601116 0.09601693 0.09082163 0.09082163]
 [0.22782374 0.23590533 0.23936887 0.23436598 0.23725226 0.23436598
  0.23359631 0.23398114 0.23994612 0.22724649]
 [0.08581874 0.08601116 0.08062344 0.07504329 0.08427939 0.07792957
  0.07234943 0.07465846 0.08139311 0.08466423]]

[0.08876275 0.23438522 0.08027708]

[0.00382333 0.00401363 0.00477012]


**특성 중요도를 계산하기 위해서는 permutation_importance()** 함수를 사용.

이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰.(**훈련 세트뿐만 아니라 테스트 세트, 사이킷런에서 제공하는 추정기 모델에도 모두 사용 가능**)

 permutation_importance() 함수는
 - 반복하여 얻은 특성 중요도 -> importances
 - 평균 -> importances_mean
 - 표준 편차 -> iumportances_std
 
 에 지정된다.

In [ ]:
print(np.mean(result.importances))

0.13447501763838116



In [ ]:
result = permutation_importance(hgb, test_input, test_target,                      # 테스트 세트에도 적용이 가능하다.
                                n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

히스토램 기반 그레이디언트 부스팅의 회귀 버전은 HistGradientBoostingRegressor 클래스 이다.

## [XGBoost](https://xgboost.readthedocs.io/en/latest)

- 사이킷런의 cross_validate() 함수와 함께 사용 가능.
- tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅으로 사용 가능.

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)

scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


## [LightGBM](https://lightgbm.readthedocs.io/en/latest)

- 마이크로소프트에서 제작.

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


# 정리

- **랜덤 포레스트** : **부트스트랩** 샘플 사용. 대표 앙상블 학습 알고리즘.
- **엑스트라 트리** : 결정 트리의 노드를 **랜덤하게 분할**
- **그레이디언트 부스팅** : 이전 트리의 손실을 보완하는 식으로 **얕은 결정 트리를 연속하여 추가**함.
- **히스토그램 기반 그레이디언트 부스팅** : **훈련 데이터를 256개 정수 구간으로 나누어** 빠르고 높은 성능을 냄

- **RandomForestClassifier** : 랜덤 포레스트 분류 클래스.
    - **n_estimators** : **트리의 개수를 지정**. 기본값은 100.
    - **critierion** : 'entropy'를 선택하여 엔트로피 **불순도**를 사용할 수 있음.
    - **max_depth** : 트리가 성장할 **최대의 깊이를 지정**. 기본값은 None. 리프 노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장.
    - **max_features** : 최적의 분할을 위해 **탐색할 특성의 개수를 지정**. 기본값은 auto로 특성 개수의 제곱근.
    - **oob_score** : **OOB 샘플**을 이용하여 훈련 모델을 평가할지 지정. 기본값은 False.
    - **n_jobs** : 병렬 실행에 사용할 **CPU 코어 수를 지정**. 기본값은 1. 만약, **-1로 지정하면 시스템에 있는 모든 코어를 사용**.


- **ExtraTreeClassifier** : 엑스트라 트리 분류 클래스.
    - **n_estimators, critierion, max_depth , min_samples_split, max_features, n_jobs** 매개변수는 랜던 포레스트와 동일.
    - **oob_score** : **OOB 샘플을 사용**하여 훈련한 모델을 평가할지 지정. 기본값은 False.

- **GradientBoostingClassifier** : 그레이디언트 부스팅 분류 클래스.
    - **loss** : **손실 함수**를 지정. 기본값은 **로지스틱 손실함수**를 의미하는 'deviance'.
    - **learning_rate** : 트리가 앙상블에 **기여하는 정도**를 조절. 기본값은 0.1.
    - **n_estimators** : **부스팅 단계**를 수행하는 트리의 개수. 기본값은 100.
    - **subsample** : **훈련 세트의 샘플 비율**을 지정. 기본값 1.0.
    - **max_depth** : 개별 **회귀 트리의 최대 깊이**. 기본값은 3.

- **HistGradientBoostingClassifier** : 히스토그램 기반 그레이디언트 부스팅 분류 클래스.
    - **learning_rate** : **학습률** 또는 **감쇠율**이라고 한다. 기본값은 0.1이며 1.0이면 감쇠가 없음.
    - **max_iter** : **부스팅 단계를 수행하는 트리의 개수**. 기본값은 100.
    - **max_bins** : **입력 데이터를 나눌 구간의 개수**. 기본값은 255이며 이보다 크게 지정할 수 없음. 여기에 1개의 구간이 누락된 값을 위해 추가된다.